# A Cando-app to setup free energy perturbation calculations using AMBER

First we do the following:

  * Load the TI-RUN package.
  * Set things up to carry out geometry optimization using Amber.

In [ ]:
*default-pathname-defaults*

In [ ]:
(ql:quickload :ti-run)

In [ ]:
(setup-default-paths)

In [ ]:
(load-atom-type-rules "ATOMTYPE_GFF.DEF")

In [ ]:
(core:getpid)

In [ ]:
source "leaprc.ff14SB.redq"

In [ ]:
source "leaprc.gaff"

In [ ]:
(list-force-fields)

In [ ]:
(start-swank)

# Start a new TI-RUN calculation

In [ ]:
(defparameter *ti-runs* (make-instance 'ti-run::ti-run-calculation))

# Import sketch - use Chemdraw

## Open chemdraw and add more functional groups if you want

In [ ]:
(png-from-file "ligands.png" )

In [ ]:
(defparameter *sk* (handler-bind ((warning #'muffle-warning))
             (with-open-file (fin (open "ligands.cdxml" :direction :input))
             (chem:make-chem-draw fin :add-hydrogens nil))))

In [ ]:
(ti-run:setup-ligands *ti-runs* *sk*)

In [ ]:
(defparameter *v* (show (ti-run:layout-ligands *ti-runs*))) *v*

In [ ]:
(defparameter *tests* (list (cons :c1 (lambda (a) (eq (chem:get-name a) :c1)))
                  (cons :c3 (lambda (a) (eq (chem:get-name a) :c3)))
                  (cons :c5 (lambda (a) (eq (chem:get-name a) :c5)))))

In [ ]:
(defparameter *pick* (chem:compile-smarts 
             "[C:6]1~[C<c1>:1]~[C:2]~[C<c3>:3]~[C:4]~[C<c5>:5]1" :tests *tests*))

In [ ]:
(show (ti-run:molecule (find-if (lambda (x) (eq (ti-run:name x) :AA)) (ti-run:ligands *ti-runs*))))

# Load the lysozyme PDB file

In [ ]:
(defparameter *lysozyme* (load-pdb "181L_mod.pdb"))

In [ ]:
(cando:build-unbuilt-hydrogens *lysozyme*)

In [ ]:
(simple-build-unbuilt-atoms *lysozyme*)

In [ ]:
(defparameter *v* (show *lysozyme*)) *v*

In [ ]:
(nglv::add-representation *v* "ball+stick" :selection "all")

In [ ]:
(pushnew *lysozyme* (ti-run:receptors *ti-runs*))

# Load the ligands

In [ ]:
*default-pathname-defaults*

In [ ]:
(load-off "phen.lib")
(load-off "benz.lib")

In [ ]:
(defparameter *ligs* (load-pdb "bnz_phn.pdb"))

In [ ]:
(simple-build-unbuilt-atoms *ligs*)
(build-unbuilt-hydrogens *ligs*)

In [ ]:
(show *ligs*)

## Print the fixed atoms of the template ligand

In [ ]:
(alexandria:hash-table-alist (ti-run:pattern-atoms *pick* *ligs*))

# Use the GAFF force field to build chemically reasonable structures of candidate ligands

In [ ]:
(ti-run:build-ligands *ti-runs*)

In [ ]:
(mapcar #'ti-run:molecule (ti-run:ligands *ti-runs*))

In [ ]:
(show (ti-run:layout-ligands *ti-runs* :accessor 'ti-run::molecule))

# Pose the new ligands onto the template ligand

In [ ]:
(ti-run::pose-ligands-using-pattern *ti-runs* *pick* *ligs*)

In [ ]:
(defparameter vreceptor (show *lysozyme*)) vreceptor

In [ ]:
(defparameter *moveable-agg* (chem:make-aggregate))
(chem:add-matter *moveable-agg* (ti-run::molecule (second (ti-run:ligands *ti-runs*))))
(defparameter *struct* (make-instance 'cando-structure :matter *moveable-agg*))
(nglv::add-structure vreceptor *struct*)

# Define the pairs of compounds between which we want to carry out free energy perturbation calculations

In [ ]:
(ti-run:build-job-nodes *ti-runs*)

In [ ]:
(ti-run::connect-job-nodes *ti-runs* :simple :connections 2 :stages 3 :windows 11)

In [ ]:
(ti-run:jobs *ti-runs*)

In [ ]:
(graph::save-graph *ti-runs*)

# Generate the calculations

In [ ]:
(defparameter *work-list* (ti-run::generate-jobs *ti-runs*))

In [ ]:
*work-list*

In [ ]:
(with-open-file (sout "/tmp/graph.dot" :direction :output)
    (ti-rundot::draw-graph-stream (list *work-list*) sout))

In [ ]:
(ext:system "dot -Tpdf -O /tmp/graph.dot")

In [ ]:
(ext:system "open -n /tmp/graph.dot.pdf")